In [4]:
%pip install ccxt pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import ccxt
import time
import pandas as pd

# --- CONFIGURACIÓN ---
symbol = 'BTC/USDT'  # El par que quieres analizar
inversion_simulada = 1000  # USD
fee_taker = 0.001  # 0.1% (Estándar en Binance/OKX sin descuentos)

print(f"--- INICIANDO ANÁLISIS DE VIABILIDAD: {symbol} ---")
print("Lentes: Ingeniería de Redes & Financiera\n")

# 1. Inicialización de los Exchanges (Modo Público para velocidad)
exchanges = {
    'binance': ccxt.binance(),
    'okx': ccxt.okx()
}

results = []

def get_market_data(exchange_name, exchange_obj):
    try:
        start_time = time.time()
        ticker = exchange_obj.fetch_ticker(symbol)
        end_time = time.time()
        
        latency_ms = (end_time - start_time) * 1000
        
        return {
            'exchange': exchange_name,
            'bid': ticker['bid'],   # Precio más alto que alguien paga (Venta)
            'ask': ticker['ask'],   # Precio más bajo que alguien vende (Compra)
            'latency_ms': latency_ms
        }
    except Exception as e:
        print(f"Error en {exchange_name}: {e}")
        return None

# 2. Captura de Datos (Simulamos 5 iteraciones para promediar)
print("Capturando datos de mercado y latencia...")
for _ in range(5):
    data_binance = get_market_data('binance', exchanges['binance'])
    data_okx = get_market_data('okx', exchanges['okx'])
    
    if data_binance and data_okx:
        # --- LENTE FINANCIERA: CÁLCULO DE ARBITRAJE ---
        
        # Opción 1: Comprar Binance -> Vender OKX
        # Costo Compra = Precio Ask Binance * (1 + fee)
        costo_compra_b = data_binance['ask'] * (1 + fee_taker)
        # Ingreso Venta = Precio Bid OKX * (1 - fee)
        ingreso_venta_o = data_okx['bid'] * (1 - fee_taker)
        profit_pct_1 = ((ingreso_venta_o - costo_compra_b) / costo_compra_b) * 100
        
        # Opción 2: Comprar OKX -> Vender Binance
        costo_compra_o = data_okx['ask'] * (1 + fee_taker)
        ingreso_venta_b = data_binance['bid'] * (1 - fee_taker)
        profit_pct_2 = ((ingreso_venta_b - costo_compra_o) / costo_compra_o) * 100
        
        results.append({
            'Latencia_Binance': f"{data_binance['latency_ms']:.1f}ms",
            'Latencia_OKX': f"{data_okx['latency_ms']:.1f}ms",
            'Spread_B->O (%)': f"{profit_pct_1:.4f}%",
            'Spread_O->B (%)': f"{profit_pct_2:.4f}%",
            '¿Rentable?': "SÍ" if profit_pct_1 > 0 or profit_pct_2 > 0 else "NO"
        })
    time.sleep(1) # Pausa breve para no saturar

results = results[1:]
# 3. Presentación de Resultados
df = pd.DataFrame(results)
print("\n--- RESULTADOS DE LA SIMULACIÓN ---")
print(df)

# Análisis final
avg_latency_binance = sum([float(x['Latencia_Binance'].replace('ms','')) for x in results]) / len(results)
avg_latency_okx = sum([float(x['Latencia_OKX'].replace('ms','')) for x in results]) / len(results)

print(f"\n--- DIAGNÓSTICO DEL EXPERTO ---")
print(f"Latencia Promedio Binance: {avg_latency_binance:.1f} ms")
print(f"Latencia Promedio OKX: {avg_latency_okx:.1f} ms")

umbral_rentabilidad = fee_taker * 2 * 100 # Necesitas cubrir 2 fees (0.2%)
print(f"Umbral mínimo de spread necesario (Break-even): {umbral_rentabilidad}%")

if avg_latency_binance > 200 or avg_latency_okx > 200:
    print(">> ADVERTENCIA CRÍTICA: La latencia es alta. El precio cambiará antes de que tu orden llegue.")
else:
    print(">> LATENCIA ACEPTABLE: Tienes luz verde técnica para optimizar.")

--- INICIANDO ANÁLISIS DE VIABILIDAD: BTC/USDT ---
Lentes: Ingeniería de Redes & Financiera

Capturando datos de mercado y latencia...

--- RESULTADOS DE LA SIMULACIÓN ---
  Latencia_Binance Latencia_OKX Spread_B->O (%) Spread_O->B (%) ¿Rentable?
0          242.3ms      301.5ms        -0.1937%        -0.2060%         NO
1          326.9ms      305.4ms        -0.1937%        -0.2060%         NO
2          230.7ms      295.8ms        -0.1937%        -0.2060%         NO
3          663.7ms      327.5ms        -0.1937%        -0.2060%         NO

--- DIAGNÓSTICO DEL EXPERTO ---
Latencia Promedio Binance: 365.9 ms
Latencia Promedio OKX: 307.6 ms
Umbral mínimo de spread necesario (Break-even): 0.2%
>> ADVERTENCIA CRÍTICA: La latencia es alta. El precio cambiará antes de que tu orden llegue.


# nota 
Es imporante notar que llegamos casi a los 200ms de latencia usando conexiones estandares, con una mejor ubicacion de la conexion podemos facilmente bajarlo 
